---
# **LAB 6 - Ottimizzazione ed efficienza**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

GPU computing notebooks download (from github)

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

NVCC Plugin for Jupyter notebook

In [ ]:
%cd GPUcomputing/utils/nvcc4jupyter-master/
!!python3 -m build
%load_ext nvcc4jupyter
%cd /content/

# ✅ Occupancy Calculator - DeviceQuery

In [ ]:
%%cuda
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"

int main(void) {

	printf("\nCUDA Device Query (Runtime API) version (CUDART static linking)\n\n");
	int deviceCount = 0;
	CHECK(cudaGetDeviceCount(&deviceCount));

	// This function call returns 0 if there are no CUDA capable devices.
	if (deviceCount == 0)
		printf("There are no available device(s) that support CUDA\n");
	else
		printf("Detected %d CUDA Capable device(s)\n", deviceCount);

	int dev, driverVersion = 0, runtimeVersion = 0;

	for (dev = 0; dev < deviceCount; ++dev) {
		cudaSetDevice(dev);
		cudaDeviceProp deviceProp;
		cudaGetDeviceProperties(&deviceProp, dev);

		printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);

		cudaDriverGetVersion(&driverVersion);
		cudaRuntimeGetVersion(&runtimeVersion);

		printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
				driverVersion / 1000, (driverVersion % 100) / 10,
				runtimeVersion / 1000, (runtimeVersion % 100) / 10);

		printf("  GPU arch name:                                 %s\n",
						_ConvertSMVer2ArchName(deviceProp.major, deviceProp.minor));

		printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
				deviceProp.major, deviceProp.minor);

		printf("  Total amount of global memory:                 %.0f MBytes (%llu bytes)\n",
				(float) deviceProp.totalGlobalMem / 1048576.0f,
				(unsigned long long) deviceProp.totalGlobalMem);

		printf("  (%2d) Multiprocessors, (%3d) CUDA Cores/MP:     %d CUDA Cores\n",
						deviceProp.multiProcessorCount,
						_ConvertSMVer2Cores(deviceProp.major, deviceProp.minor),
						_ConvertSMVer2Cores(deviceProp.major, deviceProp.minor) *
								deviceProp.multiProcessorCount);

		printf("  GPU Max Clock rate:                            %.0f MHz (%0.2f GHz)\n",
				deviceProp.clockRate * 1e-3f, deviceProp.clockRate * 1e-6f);

		printf("  Memory Clock rate:                             %.0f Mhz\n", deviceProp.memoryClockRate * 1e-3f);
		printf("  Memory Bus Width:                              %d-bit\n", deviceProp.memoryBusWidth);
		if (deviceProp.l2CacheSize)
			printf("  L2 Cache Size:                                 %d bytes\n", deviceProp.l2CacheSize);

		printf("  Maximum Texture Dimension Size (x,y,z)         1D=(%d), 2D=(%d, %d), 3D=(%d, %d, %d)\n",
				deviceProp.maxTexture1D, deviceProp.maxTexture2D[0],
				deviceProp.maxTexture2D[1], deviceProp.maxTexture3D[0],
				deviceProp.maxTexture3D[1], deviceProp.maxTexture3D[2]);

		printf("  Maximum Layered 1D Texture Size, (num) layers  1D=(%d), %d layers\n",
				deviceProp.maxTexture1DLayered[0],
				deviceProp.maxTexture1DLayered[1]);

		printf("  Maximum Layered 2D Texture Size, (num) layers  2D=(%d, %d), %d layers\n",
				deviceProp.maxTexture2DLayered[0],
				deviceProp.maxTexture2DLayered[1],
				deviceProp.maxTexture2DLayered[2]);

		printf("  Total amount of constant memory                %lu bytes\n",
				deviceProp.totalConstMem);
		printf("  Total amount of shared memory per block        %lu bytes\n",
				deviceProp.sharedMemPerBlock);
		printf("  Total number of registers available per block  %d\n",
				deviceProp.regsPerBlock);
		printf("  Warp size                                      %d\n",
				deviceProp.warpSize);
		printf("  Maximum number of threads per multiprocessor   %d\n",
				deviceProp.maxThreadsPerMultiProcessor);
		printf("  Maximum number of threads per block            %d\n",
				deviceProp.maxThreadsPerBlock);
		printf("  Max dimension size of a thread block (x,y,z)  (%d, %d, %d)\n",
				deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1],
				deviceProp.maxThreadsDim[2]);
		printf("  Max dimension size of a grid size    (x,y,z)  (%d, %d, %d)\n",
				deviceProp.maxGridSize[0], deviceProp.maxGridSize[1],
				deviceProp.maxGridSize[2]);
		printf("  Maximum memory pitch                           %lu bytes\n",
				deviceProp.memPitch);
		printf("  Texture alignment                              %lu bytes\n",
				deviceProp.textureAlignment);
		printf("  Concurrent copy and kernel execution           %s with %d copy engine(s)\n",
				(deviceProp.deviceOverlap ? "Yes" : "No"),
				deviceProp.asyncEngineCount);
		printf("  Run time limit on kernels                      %s\n",
				deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
		printf("  Integrated GPU sharing Host Memory             %s\n",
				deviceProp.integrated ? "Yes" : "No");
		printf("  Support host page-locked memory mapping        %s\n",
				deviceProp.canMapHostMemory ? "Yes" : "No");
		printf("  Alignment requirement for Surfaces             %s\n",
				deviceProp.surfaceAlignment ? "Yes" : "No");
		printf("  Device has ECC support                         %s\n",
				deviceProp.ECCEnabled ? "Enabled" : "Disabled");

		printf("  Device supports Unified Addressing (UVA):      %s\n",
				deviceProp.unifiedAddressing ? "Yes" : "No");
		printf("  Device PCI Domain ID / Bus ID / location ID:   %d / %d / %d\n",
				deviceProp.pciDomainID, deviceProp.pciBusID,
				deviceProp.pciDeviceID);
	}
	return 0;
}


Several API functions exist to assist programmers in choosing thread block size and cluster size based on register and shared memory requirements.

- `cudaOccupancyMaxActiveBlocksPerMultiprocessor`, is the occupancy calculator API that provides an **occupancy prediction based on the block size and shared memory usage** of a kernel. This function reports occupancy in terms of the number of concurrent thread blocks per multiprocessor.
Note that this value can be converted to other metrics. Multiplying by the number of warps per block yields the number of concurrent warps per multiprocessor; further dividing concurrent warps by max warps per multiprocessor gives the occupancy as a percentage.
- `cudaOccupancyMaxPotentialBlockSize` and `cudaOccupancyMaxPotentialBlockSizeVariableSMem`, are the occupancy-based launch configurator APIs, **heuristically calculate an execution configuration that achieves the maximum multiprocessor-level occupancy.**
- `cudaOccupancyMaxActiveClusters`, can provided occupancy prediction based on the cluster size, block size and shared memory usage of a kernel. This function reports occupancy in terms of number of max active clusters of a given size on the GPU present in the system.



---


The following code sample calculates the occupancy of MyKernel. It then reports the occupancy level with the **ratio between concurrent warps versus maximum warps per multiprocessor**.

In [ ]:
%%cuda_group_save --name "occupancy.cu" --group "lez6"
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"


// Device code
__global__ void MyKernel(int *d, int *a, int *b) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  d[idx] = a[idx] * b[idx];
}

// Host code
int main() {
  device_feat();    // device feats
  int numBlocks;    // number of of active blocks

  // These variables are used to convert occupancy to warps
  int device;
  cudaDeviceProp prop;
  int activeWarps;
  int maxWarps;

  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;

  for (int blockSize = 16; blockSize <= 1024; blockSize*=2) {
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, MyKernel, blockSize, 0);
    activeWarps = numBlocks * blockSize / prop.warpSize;
    double occup = (double)activeWarps / maxWarps * 100;
    printf("blockSize = %4d <-> Occupancy [numBlocks = %2d,  activeWarps = %2d]:\t%2.2f%%\n", blockSize, numBlocks, activeWarps, occup);
  }
  return 0;
}

↩ **Run...**

In [ ]:
!nvcc -arch=sm_75 src/lez6/occupancy.cu -o occupancy
!./occupancy



---

The following code sample configures an occupancy-based kernel launch of MyKernel according to the user input.


In [ ]:
%%cuda_group_save --name "occupancy.cu" --group "lez6"
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"


// Device code
__global__ void MyKernel(float *array, int arrayCount) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  if (idx < arrayCount)
    array[idx] *= array[idx];
}

__global__ void MyKernel1(int *d, int *a, int *b) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  d[idx] = a[idx] * b[idx];
}

// occupancy calculator
int occupancy(float *array, int arrayCount) {
  int blockSize;      // The launch configurator returned block size
  int minGridSize;    // The minimum grid size needed to achieve the
                      // maximum occupancy for a full device launch
  int gridSize;       // The actual grid size needed, based on input size

  cudaOccupancyMaxPotentialBlockSize(&minGridSize, &blockSize, (void*)MyKernel, 0, arrayCount);

  // Round up according to array size
  gridSize = (arrayCount + blockSize - 1) / blockSize;
  printf("blockSize = %4d, gridSize = %d, minGridSize = %2d\n", blockSize, gridSize, minGridSize);

  MyKernel<<<gridSize, blockSize>>>(array, arrayCount);
  cudaDeviceSynchronize();

  // compute occupancy
  int device;
  int numBlocks;
  cudaDeviceProp prop;
  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  int maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;
  cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, MyKernel, blockSize, 0);
  int activeWarps = numBlocks * blockSize / 32;
  double occup = (double)activeWarps / maxWarps * 100;
  printf("Occupancy [blockSize = %4d, activeWarps = %2d]:\t%2.2f%%\n", blockSize, activeWarps, occup);

  return 0;
}

// MAIN
int main(void) {
  device_feat();    // device feats
  int N = 2<<20;
  size_t size = N * sizeof(float);

  // Allocate input vectors h_A and h_B in host memory
  float* h_A = (float*)malloc(size);
  float* h_B = (float*)malloc(size);
  float* h_C = (float*)malloc(size);

  // Allocate vectors in device memory
  float* d_A;
  cudaMalloc(&d_A, size);
  float* d_B;
  cudaMalloc(&d_B, size);
  float* d_C;
  cudaMalloc(&d_C, size);

  // Copy vectors from host memory to device memory
  cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

  // Invoke kernel
  occupancy(d_A, N);

  // Free device memory
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

  return 0;
}

↩ **Run...**

In [ ]:
!nvcc -arch=sm_75  src/lez6/occupancy.cu -o occupancy
!./occupancy

Apply the same calculator to the kernel:

```
__global__ void blockParReduce1(int *in, int *out, ulong n)
```



In [ ]:
%%cuda_group_save --name "occupancy.cu" --group "lez6"
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"

__global__ void blockParReduce1(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

// MAIN
int main(void) {
  int numBlocks;        // Occupancy in terms of active blocks

  // These variables are used to convert occupancy to warps
  int device;
  cudaDeviceProp prop;
  int activeWarps;
  int maxWarps;

  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;

  for (int blockSize = 16; blockSize <= 1024; blockSize*=2) {
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, blockParReduce1, blockSize, 0);
    activeWarps = numBlocks * blockSize / prop.warpSize;
    double occup = (double)activeWarps / maxWarps * 100;
    printf("Occupancy [blockSize = %4d, activeWarps = %2d]:\t%2.2f%%\n", blockSize, activeWarps, occup);
  }
  return 0;
}

↩ **Run...**

In [ ]:
!nvcc -arch=sm_75 --ptxas-options=-v src/lez6/occupancy.cu -o occupancy
!./occupancy

Apply the same calculator to the kernel:

```
__global__ void blockParReduce_SMEM(int *in, int *out, ulong n)
```


In [ ]:
%%cuda_group_save --name "occupancy.cu" --group "lez6"
#include <stdlib.h>
#include <stdio.h>
#include "/content/GPUcomputing/utils/helper_string.h"
#include "/content/GPUcomputing/utils/helper_cuda.h"
#include "/content/GPUcomputing/utils/common.h"

#define SMEM_DIM 1024

/*
    This version uses sequential addressing -- no divergence or bank conflicts.
*/
__global__ void blockParReduce_SMEM(int *in, int *out, ulong n) {

	// shared mem
	__shared__ int smem[SMEM_DIM];

	unsigned int tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// load shared mem
	smem[tid] = (idx < n) ? in[idx] : 0;

	// synchronize within threadblock
	__syncthreads();

	// do reduction in shared mem
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
		if (tid < stride)
			smem[tid] += smem[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = smem[0];
}


// MAIN
int main(void) {
  int numBlocks;        // Occupancy in terms of active blocks

  // These variables are used to convert occupancy to warps
  int device;
  cudaDeviceProp prop;
  int activeWarps;
  int maxWarps;

  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;

  for (int blockSize = 16; blockSize <= 1024; blockSize*=2) {
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, blockParReduce_SMEM, blockSize, 0);
    activeWarps = numBlocks * blockSize / prop.warpSize;
    double occup = (double)activeWarps / maxWarps * 100;
    printf("Occupancy [blockSize = %4d, activeWarps = %2d]:\t%2.2f%%\n", blockSize, activeWarps, occup);
  }
  return 0;
}

↩ **Run...**

In [ ]:
!nvcc -arch=sm_75 --ptxas-options=-v src/lez6/occupancy.cu -o occupancy
!./occupancy

In [ ]:
!nvcc -arch=sm_75 --ptxas-options=-v src/lez6/occupancy.cu

# ✅ Profiling

### Profilazione dettagliata del filtro PPM blurring implemenato in  `ppm_blurGPU.cu`

In [ ]:
%%cuda_group_save --name "ppm_blurGPU.cu" --group "lez6"

#include <stdio.h>
#include <stdlib.h>
#include "ppm.h"
#include "../../GPUcomputing/utils/common.h"

/*
 * Set pel (pixel element) in ppm image.
 */
 __device__ void ppm_setGPU(PPM ppm, int x, int y, pel c) {
  int i = x + y*ppm.width;
  ppm.image[3*i] = c.r;
  ppm.image[3*i + 1] = c.g;
  ppm.image[3*i + 2] = c.b;
}

/*
* Get pel (pixel element) from ppm image.
*/
__device__ pel ppm_getGPU(PPM ppm, int x, int y) {
  pel p;
  int i = x + y*ppm.width;
  p.r = ppm.image[3*i];
  p.g = ppm.image[3*i + 1];
  p.b = ppm.image[3*i + 2];
  return p;
}

/*
 * Kernel 2D for PPM image blurring
 */
__global__ void ppm_blurGPU(PPM ppm, PPM ppm1, int MASK_SIZE) {

  int x = blockIdx.x * blockDim.x + threadIdx.x;
  int y = blockIdx.y * blockDim.y + threadIdx.y;

  if(x < ppm.width && y < ppm.height) {
    float R=0, G=0, B=0;
    int numPixels = 0;
    int RADIUS = MASK_SIZE/2;
    for(int r = -RADIUS; r < RADIUS; ++r) {
        for(int c = -RADIUS; c < RADIUS; ++c) {
            int row = y + r;
            int col = x + c;
            if(row > -1 && row < ppm.height && col > -1 && col < ppm.width) {
                int i = col + row*ppm.width;
                R += ppm.image[3*i];
                G += ppm.image[3*i + 1];
                B += ppm.image[3*i + 2];
                numPixels++;
            }
        }
    }
    int i = x + y*ppm.width;
    ppm1.image[3*i]     = (color)(R/numPixels);
    ppm1.image[3*i + 1] = (color)(G/numPixels);
    ppm1.image[3*i + 2] = (color)(B/numPixels);
  }
}

/*
 * MAIN
 */
int main(int argc, char **argv) {

  // PPM images
  PPM *ppm, *ppm1, *ppm2;  // Where images are stored in CPU
  PPM ppm_d, ppm1_d;	     // Where images are stored in GPU

  // load a PPM image from file
  char path[] = "GPUcomputing/images/dog.ppm";
  ppm = ppm_load(path);
  ppm1 = ppm_copy(ppm);
  uint WIDTH = ppm->width;
  uint HEIGHT = ppm->height;
  printf("PPM image size (w x h): %d x %d\n", WIDTH, HEIGHT);

  // set main params
  size_t nBytes= WIDTH * HEIGHT * sizeof(pel);
  ppm_d.width = WIDTH;
  ppm_d.height = HEIGHT;
  ppm_d.maxval = ppm->maxval;
  int MASK_SIZE = 21;

  // Allocate GPU buffer for the input and output images
  CHECK(cudaMalloc(&ppm_d.image, nBytes));
  CHECK(cudaMalloc(&ppm1_d.image, nBytes));

  // copy image from CPU to GPU
  CHECK(cudaMemcpy(ppm_d.image, ppm->image, nBytes, cudaMemcpyHostToDevice));

  // invoke kernels (define grid and block sizes)
  uint dimBlock = 16;
  dim3 block(dimBlock, dimBlock);
  dim3 grid((WIDTH + block.x - 1) / block.x, (HEIGHT + block.y - 1) / block.y);

  double start = seconds();
  ppm_blurGPU <<<grid, block>>> (ppm_d, ppm1_d, MASK_SIZE);
  CHECK(cudaDeviceSynchronize());
  double stopGPU = seconds() - start;

  // copy image from GPU to CPU
  CHECK(cudaMemcpy(ppm1->image, ppm1_d.image, nBytes, cudaMemcpyDeviceToHost));
  ppm_write(ppm1, "ppm_blurredGPU.ppm");

  // check results with CPU
  ppm2 = ppm_make(ppm->width, ppm->height, (pel){0,0,0});
  start = seconds();
  ppm_blur(ppm, ppm2, MASK_SIZE);
  double stopCPU = seconds() - start;
  ppm_write(ppm2, "ppm_blurredCPU.ppm");
  printf("PPM images are %s\n", ppm_equal(ppm1, ppm2) ? "equal" : "not equal");

  // free device memory
  CHECK(cudaFree(ppm_d.image));
  CHECK(cudaFree(ppm1_d.image));

  // times & speedup
  printf("CPU elapsed time: %.4f (msec) \n", stopCPU*1000);
  printf("CPU elapsed time: %.4f (msec) - Speedup %.1f\n", stopGPU*1000, stopCPU/stopGPU);

  return (EXIT_SUCCESS);
}


↩ **Run...**

In [ ]:
!nvcc -arch=sm_75 src/lez6/ppm_blurGPU.cu -o blur -I GPUcomputing/utils/PPM  GPUcomputing/utils/PPM/ppm.cpp
!./blur

↩ **Profiling...**

In [ ]:
!ncu -f -o ppm_blurGPU_T4_profiling.ncu-rep blur

In [ ]:
!ncu -i /content/ppm_blurGPU_T4_profiling.ncu-rep

### Profilazione dettagliata del filtro di convoluzione implemenato in `conv2D.cu`

In [ ]:
%%cuda_group_save --name "conv2D.cu" --group "lez6"
#include "/content/GPUcomputing/utils/common.h"

#define BLOCK_SIZE   16
#define MASK_SIZE    21
#define TILE_SIZE    (BLOCK_SIZE + MASK_SIZE - 1)

typedef struct {
   int width;
   int height;
   float* elements;
 } Matrix;

// Function declarations
 void conv2D_host(Matrix A, Matrix B, Matrix M);
__global__ void conv2D_basic(Matrix A, Matrix B, Matrix M);

 /*
  * 2D convolution using shared memory
  *   A: input matrix
  *   B: output matrix
  *   M: convolution mask matrix
 */
__global__ void conv2D(Matrix A, Matrix B, Matrix M) {

   int x = blockIdx.x * blockDim.x + threadIdx.x; // Column index of matrix A
   int y = blockIdx.y * blockDim.y + threadIdx.y; // Row index of matrix A

   int tile_size = BLOCK_SIZE + MASK_SIZE - 1;
   int radius = MASK_SIZE / 2;

   // Allocate shared memory
   __shared__ float smem[TILE_SIZE][TILE_SIZE];

   // Load data into shared memory
   for (int row = 0; row <= tile_size/blockDim.y; row++) {
      for (int col = 0; col <= tile_size/blockDim.x; col++) {
         int row_data = y + blockDim.y * row - radius;   // input data index row
         int col_data = x + blockDim.x * col - radius;   // input data index column
         int row_smem = threadIdx.y + blockDim.y * row;  // mask index row
         int col_smem = threadIdx.x + blockDim.x * col;  // mask index column

         // Check valid range for smem and data
         if (row_smem < tile_size && col_smem < tile_size) {
            if (row_data >= 0 && row_data < A.height && col_data >= 0 && col_data < A.width) {
               smem[row_smem][col_smem] = A.elements[row_data * A.width + col_data];
            } else {
               smem[row_smem][col_smem] = 0.0f;
            }
         }
      }
   }

   // Synchronize threads
   __syncthreads();

   // Apply convolution
   float sum = 0.0f;
   for (int i = 0; i < MASK_SIZE; i++) {
      for (int j = 0; j < MASK_SIZE; j++) {
         int r = threadIdx.y + i;
         int c = threadIdx.x + j;
         if (r >= 0 && r < tile_size && c >= 0 && c < tile_size) {
            sum += smem[r][c] * M.elements[i * MASK_SIZE + j];
         }
      }
   }

   // Write output
   if (y < A.height && x < A.width) {
      B.elements[y * B.width + x] = sum;
   }
}

/*
 * Main function
 */
int main(void) {
   // define matrices and params
   int block_size = BLOCK_SIZE;
   int mask_size = MASK_SIZE;
   int width = 256* block_size, height = 256* block_size;
   Matrix A, B, H, M;
   A.width = width; A.height = height;
   B.width = width; B.height = height;
   M.width = mask_size; M.height = mask_size;
   H.width = width; H.height = height;
   A.elements = (float *)malloc(width * height * sizeof(float));
   B.elements = (float *)malloc(width * height * sizeof(float));
   M.elements = (float *)malloc(mask_size * mask_size * sizeof(float));
   H.elements = (float *)malloc(width * height * sizeof(float));

   // Initialize A, B, M
   // print data sizes
   printf("Data matrix A: %d x %d\n", width, height);
   printf("Mask matrix M: %d x %d\n", mask_size, mask_size);
   for (int i = 0; i < width * height; i++) {
      A.elements[i] = 1.0f;
      B.elements[i] = 0.0f;
   }
   for (int i = 0; i < mask_size * mask_size; i++) {
      M.elements[i] = 1.0f;
   }

   // Allocate device memory
   Matrix d_A, d_B, d_M;
   d_A.width = A.width; d_A.height = A.height;
   d_B.width = B.width; d_B.height = B.height;
   d_M.width = M.width; d_M.height = M.height;
   CHECK(cudaMalloc(&d_A.elements, width * height * sizeof(float)));
   CHECK(cudaMalloc(&d_B.elements, width * height * sizeof(float)));
   CHECK(cudaMalloc(&d_M.elements, mask_size * mask_size * sizeof(float)));

   // Copy data to device
   CHECK(cudaMemcpy(d_A.elements, A.elements, width * height * sizeof(float), cudaMemcpyHostToDevice));
   CHECK(cudaMemcpy(d_M.elements, M.elements, mask_size * mask_size * sizeof(float), cudaMemcpyHostToDevice));

   /***********************************************************/
	/*                    conv2D on host                       */
	/***********************************************************/
   printf("\nCPU procedure...\n");
	double start = seconds();
	conv2D_host(A, H, M);
	double stopCPU = seconds() - start;
   printf("   Host elapsed time: %f\n", stopCPU);

   /***********************************************************/
	/*                    GPU naive conv2D                     */
	/***********************************************************/
   printf("\nGPU naive conv2D...\n");
   dim3 dimBlock(block_size, block_size);
   dim3 dimGrid((width + block_size - 1) / block_size, (height + block_size - 1) / block_size);
   start = seconds();
   conv2D_basic<<<dimGrid, dimBlock>>>(d_A, d_B, d_M);
   CHECK(cudaDeviceSynchronize());
   double stopGPU = seconds() - start;
   printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, stopCPU / stopGPU);

   // Copy data back to host
   CHECK(cudaMemcpy(B.elements, d_B.elements, width * height * sizeof(float), cudaMemcpyDeviceToHost));

   // check results
   for (int i = 0; i < width; i++) {
      for (int j = 0; j < height; j++) {
         if (B.elements[j * width + i] != H.elements[j * width + i]) {
            printf("Error at B[%d][%d] = %f\n", i, j, B.elements[j * width + i]);
         }
      }
   }

   // zero out B in device
   CHECK(cudaMemset(d_B.elements, 0, width * height * sizeof(float)));

   /***********************************************************/
	/*                  GPU conv2D wih smem                    */
	/***********************************************************/
   printf("\nGPU conv2D with smem...\n");
   start = seconds();
   conv2D<<<dimGrid, dimBlock>>>(d_A, d_B, d_M);
   CHECK(cudaDeviceSynchronize());
   stopGPU = seconds() - start;
   printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, stopCPU / stopGPU);

   // Copy data back to host
   CHECK(cudaMemcpy(B.elements, d_B.elements, width * height * sizeof(float), cudaMemcpyDeviceToHost));

   // check results
   for (int i = 0; i < width; i++) {
      for (int j = 0; j < height; j++) {
         if (B.elements[j * width + i] != H.elements[j * width + i]) {
            printf("Error at B[%d][%d] = %f\n", i, j, B.elements[j * width + i]);
         }
      }
   }

   return 0;
}

/*
 * 2D convolution on host
 */
void conv2D_host(Matrix A, Matrix B, Matrix M) {

   int radius = MASK_SIZE / 2;

   // loop through all elements in the output array
   for (int y = 0; y < A.height; y++) {
	   for (int x = 0; x < A.width; x++) {
			float sum = 0.0f;

			// compute convolution
			for (int i = 0; i < MASK_SIZE; i++) {
            for (int j = 0; j < MASK_SIZE; j++) {
               int r = y - radius + i;
					int c = x - radius + j;

					//boundary check
					if ((c >= 0) && (c < A.width) && (r >= 0) && (r < A.height)) {
						sum += A.elements[(r * A.width) + c] * M.elements[(j * MASK_SIZE) + i];
					}
				}
         }

         //store final value
         B.elements[y * B.width + x] = sum;
	   }
   }
}

/*
 * Basic kernel for 2D convolution
 */
 __global__ void conv2D_basic(Matrix A, Matrix B, Matrix M) {

	//index computation
	int x = blockIdx.x * blockDim.x + threadIdx.x;
	int y = blockIdx.y * blockDim.y + threadIdx.y;
   int radius = MASK_SIZE / 2;

   //boundary check
   if (x >= A.width && y >= A.height)  return;

   // Apply convolution
   float sum = 0.0f;
   for (int i = 0; i < MASK_SIZE; i++) {
      for (int j = 0; j < MASK_SIZE; j++) {
         int r = y - radius + i;
         int c = x - radius + j;
         if (r >= 0 && r < A.height && c >= 0 && c < A.width) {
            sum += A.elements[r * A.width + c] * M.elements[i * MASK_SIZE + j];
         }
      }
   }

   //store final value
   B.elements[y * B.width + x] = sum;
}

↩ **Run...**

In [ ]:
!nvcc -arch=sm_75 src/lez6/conv2D.cu -o conv2D
!./conv2D

↩ **Profiling...**

In [ ]:
!ncu -o conv2D_T4_profiling.ncu-rep blur

In [ ]:
!ncu -i conv2D_T4_profiling.ncu-rep

# ✅ Parallel reduction unrolling




↘️ **SOL...**

In [ ]:
%%cuda_group_save --name "preduceUnroll.cu" --group "lez6"

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include "/content/GPUcomputing/utils/common.h"

#define SMEM_DIM 1024

/*
*  Block by block parallel implementation with divergence (sequential schema)
*/
__global__ void blockParReduce1(int *in, int *out, ulong n) {

	int tid = threadIdx.x;
	int idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
*  Block by block parallel implementation without divergence (interleaved schema)
*/
__global__ void blockParReduce2(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1)  {
		if (tid < stride)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
* Using shared memory (no divergence nor bank conflicts)
*/
__global__ void blockParReduce3(int *in, int *out, ulong n) {

	// shared mem
	__shared__ int smem[SMEM_DIM];

	unsigned int tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// load shared mem
	smem[tid] = (idx < n) ? in[idx] : 0;

	// synchronize within threadblock
	__syncthreads();

	// do reduction in shared mem
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
		if (tid < stride)
			smem[tid] += smem[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = smem[0];
}

/*
*  Block by block parallel implementation using warp reduction
*/
__global__ void blockParReduce4(int *in, int *out, ulong n) {
	int tid = blockIdx.x * blockDim.x + threadIdx.x;
	int lane = threadIdx.x % warpSize;
	int wid = threadIdx.x / warpSize;

	static __shared__ int shared[32];  	// Shared mem for 32 partial sums
	int val = in[tid];  						// Each thread reads one element

	// 1° warp-shuffle reduction
	for (int offset = warpSize/2; offset > 0; offset /= 2)
		val += __shfl_down_sync(0xffffffff, val, offset, 32);

	if (lane==0) shared[wid] = val; 	// Write reduced value to shared memory

	__syncthreads();          			// Wait for all partial reductions

	// hereafter, just warp 0 (final warp-shuffle reduction)
	if (wid == 0){
		int val = shared[lane];

		for (int offset = warpSize/2; offset > 0; offset >>= 1)
			val += __shfl_down_sync(0xffffffff, val, offset);

			// write result for this block to global mem
		if (threadIdx.x == 0)
			out[blockIdx.x] = val;
	}
}

/*
*  Warp reduction using shared memory
*/
__device__ void warpReduce(volatile int *smem, unsigned int tid) {
   smem[tid] += smem[tid+32];
   smem[tid] += smem[tid+16];
   smem[tid] += smem[tid+8];
   smem[tid] += smem[tid+4];
   smem[tid] += smem[tid+2];
   smem[tid] += smem[tid+1];
 }

/*
*  Block reduction using shared memory
*/
__device__ void blockReduce(volatile int *smem, unsigned int tid) {
   // block in-place reduction in shared memory
   if (blockDim.x >= 1024 && tid < 512) smem[tid] += smem[tid + 512];
   __syncthreads();

   if (blockDim.x >= 512 && tid < 256) smem[tid] += smem[tid + 256];
   __syncthreads();

   if (blockDim.x >= 256 && tid < 128) smem[tid] += smem[tid + 128];
   __syncthreads();

   if (blockDim.x >= 128 && tid < 64)  smem[tid] += smem[tid + 64];
   __syncthreads();

   // unrolling warp
   if (tid < 32) warpReduce(smem, tid);
}

/*
*  Block by block parallel implementation using complete unroll for loop + smem
*/
__global__ void blockParReduce5(int *in, int *out, int n) {
   // SMEM for each block
   static __shared__ int smem[SMEM_DIM];

   // set thread ID
   int tid = threadIdx.x;

   // boundary check
   int idx = blockIdx.x * blockDim.x + threadIdx.x;

   if (idx >= n) return;

   // convert global data pointer to the local pointer of this block
   int *idata = in + blockIdx.x * blockDim.x;

   // set to smem by each threads
   smem[tid] = idata[tid];
   __syncthreads();

   // block in-place reduction in shared memory
   blockReduce(smem, tid);

   // write result for this block to global mem
   if (tid == 0) out[blockIdx.x] = smem[0];
}

/*
 *  Multi block parallel implementation with block and warp unrolling
 */
 __global__ void blockParReduce6(int *in, int *out, ulong n) {

   // SMEM for each block
   static __shared__ int smem[SMEM_DIM];

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x * 8 + threadIdx.x;

	// boundary check
	if (idx >= n) return;

   // group blocks unrolling
    if (idx + 7 * blockDim.x < n) {
      smem[tid] = in[idx];
      smem[tid] += in[idx + blockDim.x];
      smem[tid] += in[idx + 2 * blockDim.x];
      smem[tid] += in[idx + 3 * blockDim.x];
      smem[tid] += in[idx + 4 * blockDim.x];
      smem[tid] += in[idx + 5 * blockDim.x];
      smem[tid] += in[idx + 6 * blockDim.x];
      smem[tid] += in[idx + 7 * blockDim.x];
    }
    __syncthreads();

   // block in-place reduction in shared memory
   blockReduce(smem, tid);

   // write result for this block to global mem
   if (tid == 0) out[blockIdx.x] = smem[0];
}

/*
* MAIN: test on parallel reduction
*/
int main() {
	int *a, *b, *d_a, *d_b;
	int blockSize = 1024;             // block dim 1D
	size_t numBlock = 1024*1024;      // grid dim 1D
	size_t n = blockSize * numBlock;  // array dims
	size_t sum_CPU = 0, sum_GPU = 0;
	size_t nByte = n*sizeof(int);
	size_t mByte = numBlock * sizeof(int);
	double start, stopGPU, stopCPU, speedup;

	printf("\n****  test on parallel reduction  ****\n");

	// init
	a = (int *) malloc(nByte);
	b = (int *) malloc(mByte);
	for (size_t i = 0; i < n; i++) a[i] = 1;  // initialize a[] = 1

	CHECK(cudaMalloc(&d_a, nByte));
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc(&d_b, mByte));
	CHECK(cudaMemset(d_b, 0, mByte));

	/***********************************************************/
	/*                     CPU reduction                       */
	/***********************************************************/
	printf("  Vector length: %.2f MB\n",n/(1024.0*1024.0));
	printf("\n  CPU procedure...\n");
	start = seconds();
	for (ulong i = 0; i < n; i++)
	sum_CPU += a[i];
	stopCPU = seconds() - start;
	printf("    Elapsed time: %f (sec) \n", stopCPU);
	printf("    sum: %lu\n",sum_CPU);

	printf("\n  GPU kernels (mem required %lu bytes)\n", nByte);

	/***********************************************************/
	/*         KERNEL blockParReduce1 (divergent)              */
	/***********************************************************/
	// block by block parallel implementation with divergence
	printf("\n  Launch kernel: blockParReduce1...\n");
	start = seconds();
	blockParReduce1<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaGetLastError());
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) sum_GPU += b[i];
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

	/***********************************************************/
	/*        KERNEL blockParReduce2  (non divergent)          */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduce2...\n");
	start = seconds();
	blockParReduce2<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) sum_GPU += b[i];
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

	/***********************************************************/
	/*           KERNEL blockParReduce3 (with smem)            */
	/***********************************************************/
	// block by block parallel implementation using warp reduction
	printf("\n  Launch kernel: blockParReduce3...\n");
	start = seconds();
	blockParReduce3<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) sum_GPU += b[i];
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

	/***********************************************************/
	/*        KERNEL blockParReduce4  (warp reducton)          */
	/***********************************************************/
	// block by block parallel implementation with smem
	printf("\n  Launch kernel: blockParReduce4...\n");
	start = seconds();
	blockParReduce4<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
		//printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

   /***********************************************************/
	/*           KERNEL blockParReduce5  (unroll)              */
	/***********************************************************/
	// block by block parallel implementation with unroll + smem
	printf("\n  Launch kernel: blockParReduce5...\n");
	start = seconds();
	blockParReduce5<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
		//printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

   /***********************************************************/
	/*         KERNEL blockParReduce6  (bock unroll)           */
	/***********************************************************/
	// block by block parallel implementation with unroll + smem
	printf("\n  Launch kernel: blockParReduce6...\n");
	start = seconds();
	blockParReduce6<<<numBlock/8, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
		//printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);

	cudaFree(d_a);
	return 0;
}

In [ ]:
!nvcc -arch=sm_75  src/lez6/preduceUnroll.cu -o preduce
!./preduce

↘️ **TODO...**

Kernel privo di divergenza + unrolling:

* Introdurre warp unrolling
* Introdurre block unrolling
* Specializzare su diversi num blocchi
* Confrontare i vari kernel...




In [ ]:
%%cuda_group_save --name "preduceUnroll.cu" --group "lez6"

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include "/content/GPUcomputing/utils/common.h"

#define SMEM_DIM 1024

/*
*  Block by block parallel implementation with divergence (sequential schema)
*/
__global__ void blockParReduce1(int *in, int *out, ulong n) {

	int tid = threadIdx.x;
	int idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = 1; stride < blockDim.x; stride *= 2) {
		if ((tid % (2 * stride)) == 0)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
*  Block by block parallel implementation without divergence (interleaved schema)
*/
__global__ void blockParReduce2(int *in, int *out, ulong n) {

	uint tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// boundary check
	if (idx >= n)
		return;

	// convert global data pointer to the local pointer of this block
	int *thisBlock = in + blockIdx.x * blockDim.x;

	// in-place reduction in global memory
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1)  {
		if (tid < stride)
			thisBlock[tid] += thisBlock[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = thisBlock[0];
}

/*
* Using shared memory (no divergence nor bank conflicts)
*/
__global__ void blockParReduce3(int *in, int *out, ulong n) {

	// shared mem
	__shared__ int smem[SMEM_DIM];

	unsigned int tid = threadIdx.x;
	ulong idx = blockIdx.x * blockDim.x + threadIdx.x;

	// load shared mem
	smem[tid] = (idx < n) ? in[idx] : 0;

	// synchronize within threadblock
	__syncthreads();

	// do reduction in shared mem
	for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
		if (tid < stride)
			smem[tid] += smem[tid + stride];

		// synchronize within threadblock
		__syncthreads();
	}

	// write result for this block to global mem
	if (tid == 0)
		out[blockIdx.x] = smem[0];
}

/*
*  Block by block parallel implementation using warp reduction
*/
__global__ void blockParReduce4(int *in, int *out, ulong n) {
	int tid = blockIdx.x * blockDim.x + threadIdx.x;
	int lane = threadIdx.x % warpSize;
	int wid = threadIdx.x / warpSize;

	static __shared__ int shared[32];  	// Shared mem for 32 partial sums
	int val = in[tid];  						// Each thread reads one element

	// 1° warp-shuffle reduction
	for (int offset = warpSize/2; offset > 0; offset /= 2)
		val += __shfl_down_sync(0xffffffff, val, offset, 32);

	if (lane==0) shared[wid] = val; 	// Write reduced value to shared memory

	__syncthreads();          			// Wait for all partial reductions

	// hereafter, just warp 0 (final warp-shuffle reduction)
	if (wid == 0){
		int val = shared[lane];

		for (int offset = warpSize/2; offset > 0; offset >>= 1)
			val += __shfl_down_sync(0xffffffff, val, offset);

			// write result for this block to global mem
		if (threadIdx.x == 0)
			out[blockIdx.x] = val;
	}
}

/*
*  Warp reduction using shared memory
*/
__device__ void warpReduce(volatile int *smem, unsigned int tid) {
   smem[tid] += smem[tid+32];
   smem[tid] += smem[tid+16];
   smem[tid] += smem[tid+8];
   smem[tid] += smem[tid+4];
   smem[tid] += smem[tid+2];
   smem[tid] += smem[tid+1];
 }

/*
*  Block reduction using shared memory
*/
__device__ void blockReduce(volatile int *smem, unsigned int tid) {
   // block in-place reduction in shared memory
   if (blockDim.x >= 1024 && tid < 512) smem[tid] += smem[tid + 512];
   __syncthreads();

   if (blockDim.x >= 512 && tid < 256) smem[tid] += smem[tid + 256];
   __syncthreads();

   if (blockDim.x >= 256 && tid < 128) smem[tid] += smem[tid + 128];
   __syncthreads();

   if (blockDim.x >= 128 && tid < 64)  smem[tid] += smem[tid + 64];
   __syncthreads();

   // unrolling warp
   if (tid < 32) warpReduce(smem, tid);
}

/*
*  Block by block parallel implementation using complete unroll for loop + smem
*/
__global__ void blockParReduce5(int *in, int *out, int n) {
   // SMEM for each block
   static __shared__ int smem[SMEM_DIM];

   // set thread ID
   int tid = threadIdx.x;

   // boundary check
   int idx = blockIdx.x * blockDim.x + threadIdx.x;

   if (idx >= n) return;

   // convert global data pointer to the local pointer of this block
   int *idata = in + blockIdx.x * blockDim.x;

   // set to smem by each threads
   smem[tid] = idata[tid];
   __syncthreads();

   // block in-place reduction in shared memory
   blockReduce(smem, tid);

   // write result for this block to global mem
   if (tid == 0) out[blockIdx.x] = smem[0];
}

/*
 *  Multi block parallel implementation with block and warp unrolling
 */
 __global__ void blockParReduce6(int *in, int *out, ulong n) {

   // TODO

}

/*
* MAIN: test on parallel reduction
*/
int main() {
	int *a, *b, *d_a, *d_b;
	int blockSize = 1024;             // block dim 1D
	size_t numBlock = 1024*1024;      // grid dim 1D
	size_t n = blockSize * numBlock;  // array dims
	size_t sum_CPU = 0, sum_GPU = 0;
	size_t nByte = n*sizeof(int);
	size_t mByte = numBlock * sizeof(int);
	double start, stopGPU, stopCPU, speedup;

	printf("\n****  test on parallel reduction  ****\n");

	// init
	a = (int *) malloc(nByte);
	b = (int *) malloc(mByte);
	for (size_t i = 0; i < n; i++) a[i] = 1;  // initialize a[] = 1

	CHECK(cudaMalloc(&d_a, nByte));
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc(&d_b, mByte));
	CHECK(cudaMemset(d_b, 0, mByte));

	/***********************************************************/
	/*                     CPU reduction                       */
	/***********************************************************/
	printf("  Vector length: %.2f MB\n",n/(1024.0*1024.0));
	printf("\n  CPU procedure...\n");
	start = seconds();
	for (ulong i = 0; i < n; i++)
	sum_CPU += a[i];
	stopCPU = seconds() - start;
	printf("    Elapsed time: %f (sec) \n", stopCPU);
	printf("    sum: %lu\n",sum_CPU);

	printf("\n  GPU kernels (mem required %lu bytes)\n", nByte);

	/***********************************************************/
	/*         KERNEL blockParReduce1 (divergent)              */
	/***********************************************************/
	// block by block parallel implementation with divergence
	printf("\n  Launch kernel: blockParReduce1...\n");
	start = seconds();
	blockParReduce1<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaGetLastError());
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) sum_GPU += b[i];
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

	/***********************************************************/
	/*        KERNEL blockParReduce2  (non divergent)          */
	/***********************************************************/
	// block by block parallel implementation without divergence
	printf("\n  Launch kernel: blockParReduce2...\n");
	start = seconds();
	blockParReduce2<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU,speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) sum_GPU += b[i];
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

	/***********************************************************/
	/*           KERNEL blockParReduce3 (with smem)            */
	/***********************************************************/
	// block by block parallel implementation using warp reduction
	printf("\n  Launch kernel: blockParReduce3...\n");
	start = seconds();
	blockParReduce3<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) sum_GPU += b[i];
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

	/***********************************************************/
	/*        KERNEL blockParReduce4  (warp reducton)          */
	/***********************************************************/
	// block by block parallel implementation with smem
	printf("\n  Launch kernel: blockParReduce4...\n");
	start = seconds();
	blockParReduce4<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
		//printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

   /***********************************************************/
	/*           KERNEL blockParReduce5  (unroll)              */
	/***********************************************************/
	// block by block parallel implementation with unroll + smem
	printf("\n  Launch kernel: blockParReduce5...\n");
	start = seconds();
	blockParReduce5<<<numBlock, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
		//printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);

	// copy and reset vectors on GPU
	CHECK(cudaMemcpy(d_a, a, nByte, cudaMemcpyHostToDevice));
	CHECK(cudaMemset(d_b, 0, mByte));

   /***********************************************************/
	/*         KERNEL blockParReduce6  (bock unroll)           */
	/***********************************************************/
	// block by block parallel implementation with unroll + smem
	printf("\n  Launch kernel: blockParReduce6...\n");
	start = seconds();
	blockParReduce6<<<numBlock/8, blockSize>>>(d_a, d_b, n);
	CHECK(cudaDeviceSynchronize());
	stopGPU = seconds() - start;
	speedup = stopCPU/stopGPU;
	printf("    Elapsed time: %f (sec) - speedup %.1f\n", stopGPU, speedup);
	CHECK(cudaGetLastError());

	// memcopy D2H
	CHECK(cudaMemcpy(b, d_b, mByte, cudaMemcpyDeviceToHost));

	// check result
	sum_GPU = 0;
	for (uint i = 0; i < numBlock; i++) {
		sum_GPU += b[i];
		//printf("b[%d] = %d\n",i,b[i]);
	}
	assert(sum_GPU == n);

	cudaFree(d_a);
	return 0;
}

In [ ]:
!nvcc -arch=sm_70 src/preduceUnroll.cu -o preduceUnroll
!./preduceUnroll

# ✅ Parallelismo dinamico

In [ ]:
%%cuda --name nestedHelloWorld.cu
#include <stdio.h>

/*
 * A simple example of nested kernel launches from the GPU. Each thread displays
 * its information when execution begins, and also diagnostics when the next
 * lowest nesting layer completes.
 */

__global__ void nestedHelloWorld(int const iSize, int iDepth) {
	int tid = threadIdx.x;
	printf("Recursion=%d: Hello World from thread %d block %d\n", iDepth, tid,	blockIdx.x);

	// condition to stop recursive execution
	if (iSize == 1)
		return;

	// reduce block size to half
	int nthreads = iSize >> 1;


	// thread 0 launches child grid recursively
	if (tid == 0 && nthreads > 0) {
		nestedHelloWorld<<<gridDim.x, nthreads>>>(nthreads, ++iDepth);
		printf("-------> nested execution depth: %d\n", iDepth);
	}
}

int main(int argc, char **argv) {
	int size = 8;
	int blocksize = 8;   // initial block size
	int igrid = 2;

	size = igrid * blocksize;

	dim3 block(blocksize, 1);
	dim3 grid((size + block.x - 1) / block.x, 1);
	printf("%s Execution Configuration: grid %d block %d\n", argv[0], grid.x,
			block.x);

	nestedHelloWorld<<<grid, block>>>(block.x, 0);

	cudaDeviceReset();
	return 0;
}

↩ **Run...**

In [ ]:
!nvcc -arch=sm_70 -dc src/nestedHelloWorld.cu
!nvcc -arch=sm_70 *.o -o nestedHelloWorld
!./nestedHelloWorld


### Mandelbrot

In [ ]:
%%cuda_group_save --name "mandelbrot.cu" --group "lez6"

#include "ppm.h"
#include "/content/GPUcomputing/utils/common.h"

#define MAX_DWELL 1024
#define BS 256
#define CUT_DWELL (MAX_DWELL / 4)

/** a useful function to compute the number of threads */
int divup(int x, int y) { return x / y + (x % y ? 1 : 0); }

/** gets the color, given the dwell */
void dwell_color(int *r, int *g, int *b, int dwell);

/** a simple complex type */
struct complex {
	__host__ __device__ complex(float re, float im = 0) {
		this->re = re;
		this->im = im;
	}
	float re, im;   // real and imaginary part
};

// operator overloads for complex numbers
inline __host__ __device__ complex operator+(const complex &a, const complex &b) {
	return complex(a.re + b.re, a.im + b.im);
}

inline __host__ __device__ complex operator-(const complex &a) {
   return complex(-a.re, -a.im);
}

inline __host__ __device__ complex operator-(const complex &a, const complex &b) {
	return complex(a.re - b.re, a.im - b.im);
}

inline __host__ __device__ complex operator*(const complex &a, const complex &b) {
	return complex(a.re * b.re - a.im * b.im, a.im * b.re + a.re * b.im);
}

inline __host__ __device__ float abs2(const complex &a) {
	return a.re * a.re + a.im * a.im;
}

inline __host__ __device__ complex operator/(const complex &a, const complex &b) {
	float invabs2 = 1 / abs2(b);
	return complex((a.re * b.re + a.im * b.im) * invabs2, (a.im * b.re - b.im * a.re) * invabs2);
}


/** computes the dwell for a single pixel */
__device__ int pixel_dwell (int w, int h, complex cmin, complex cmax, int x, int y) {
	complex dc = cmax - cmin;
	float fx = (float)x / w, fy = (float)y / h;
	complex c = cmin + complex(fx * dc.re, fy * dc.im);
	int dwell = 0;
	complex z = c;
	while(dwell < MAX_DWELL && abs2(z) < 2 * 2) {
		z = z * z + c;
		dwell++;
	}
	return dwell;
}

/** computes the dwells for Mandelbrot image
		@param dwells the output array
		@param w the width of the output image
		@param h the height of the output image
		@param cmin the complex value associated with the left-bottom corner of the
		image
		@param cmax the complex value associated with the right-top corner of the
		image
 */
__global__ void mandelbrot_k (int *dwells, int w, int h, complex cmin, complex cmax) {
	// complex value to start iteration (c)
	int x = threadIdx.x + blockIdx.x * blockDim.x;
	int y = threadIdx.y + blockIdx.y * blockDim.y;
	int dwell = pixel_dwell(w, h, cmin, cmax, x, y);
	dwells[y * w + x] = dwell;
}


/** data size */
#define H (8*1024)
#define W (8*1024)
#define IMAGE_PATH "./mandelbrot.ppm"

int main(int argc, char **argv) {
	// allocate memory
	int w = W,  h = H;
	size_t dwell_sz = w * h * sizeof(int);
	int *h_dwells, *d_dwells;
	CHECK(cudaMalloc(&d_dwells, dwell_sz));
	h_dwells = (int*)malloc(dwell_sz);

	// compute the dwells, copy them back
	dim3 bs(64, 4), grid(divup(w, bs.x), divup(h, bs.y));
	double start = seconds();
	mandelbrot_k<<<grid, bs>>> (d_dwells, w, h, complex(-1.5, -1), complex(0.5, 1));
	CHECK(cudaDeviceSynchronize());
   double gpu_time = seconds() - start;
	CHECK(cudaMemcpy(h_dwells, d_dwells, dwell_sz, cudaMemcpyDeviceToHost));

   // print performance
	printf("Mandelbrot set computed in %.5lf s, at %.3lf Mpix/s\n", gpu_time, h * w * 1e-6 / gpu_time);


   // save the image
   PPM *ppm = ppm_make(w, h, (pel) {0,0,0});
   for (int y = 0; y < h; y++) {
		for (int x = 0; x < w; x++) {
			int r, g, b;
			dwell_color(&r, &g, &b, h_dwells[y * w + x]);
         ppm_set(ppm, x, y, (pel) {(color)r,(color)g,(color)b});
		}
   }
   ppm_write(ppm, IMAGE_PATH);
   printf("Image saved to %s\n", IMAGE_PATH);


	// free data
	cudaFree(d_dwells);
	free(h_dwells);
	return 0;
}

/** gets the color, given the dwell (on host) */
void dwell_color(int *r, int *g, int *b, int dwell) {
	// black for the Mandelbrot set
	if(dwell >= MAX_DWELL) {
		*r = *g = *b = 0;
	} else {
		// cut at zero
		if(dwell < 0)
			dwell = 0;
		if(dwell <= CUT_DWELL) {
			// from black to blue the first half
			*r = *g = 0;
			*b = 128 + dwell * 127 / (CUT_DWELL);
		} else {
			// from blue to white for the second half
			*b = 255;
			*r = *g = (dwell - CUT_DWELL) * 255 / (MAX_DWELL - CUT_DWELL);
		}
	}
}

↩ **Run...**

In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_75 src/lez6/mandelbrot.cu -o mandelbrot -I GPUcomputing/utils/PPM GPUcomputing/utils/PPM/ppm.cpp
!./mandelbrot

In [ ]:
%%cuda_group_save --name "mandelbrot_dyn.cu" --group "lez6"

#include "ppm.h"
#include "/content/GPUcomputing/utils/common.h"


#define CUT_DWELL (MAX_DWELL / 4)
#define MAX_DWELL 1024
/** block size along */
#define BSX 64
#define BSY 4
/** maximum recursion depth */
#define MAX_DEPTH 4
/** region below which do per-pixel */
#define MIN_SIZE 32
/** subdivision factor along each axis */
#define SUBDIV 4
/** subdivision when launched from host */
#define INIT_SUBDIV 32
#define NEUT_DWELL (MAX_DWELL + 1)
#define DIFF_DWELL (-1)

/** a useful function to compute the number of threads */
__device__ int divup(int x, int y) { return x / y + (x % y ? 1 : 0); }

/** gets the color, given the dwell */
void dwell_color(int *r, int *g, int *b, int dwell);

/** a simple complex type */
struct complex {
	__host__ __device__ complex(float re, float im = 0) {
		this->re = re;
		this->im = im;
	}
	float re, im;   // real and imaginary part
};

// operator overloads for complex numbers
inline __host__ __device__ complex operator+(const complex &a, const complex &b) {
	return complex(a.re + b.re, a.im + b.im);
}

inline __host__ __device__ complex operator-(const complex &a) {
   return complex(-a.re, -a.im);
}

inline __host__ __device__ complex operator-(const complex &a, const complex &b) {
	return complex(a.re - b.re, a.im - b.im);
}

inline __host__ __device__ complex operator*(const complex &a, const complex &b) {
	return complex(a.re * b.re - a.im * b.im, a.im * b.re + a.re * b.im);
}

inline __host__ __device__ float abs2(const complex &a) {
	return a.re * a.re + a.im * a.im;
}

inline __host__ __device__ complex operator/(const complex &a, const complex &b) {
	float invabs2 = 1 / abs2(b);
	return complex((a.re * b.re + a.im * b.im) * invabs2, (a.im * b.re - b.im * a.re) * invabs2);
}



/********************************************************/
/*                   with dyn parall                    */
/********************************************************/

/** find the dwell for the pixel */
__device__ int pixel_dwell(int w, int h, complex cmin, complex cmax, int x, int y) {
	complex dc = cmax - cmin;
	float fx = (float)x / w, fy = (float)y / h;
	complex c = cmin + complex(fx * dc.re, fy * dc.im);
	int dwell = 0;
	complex z = c;
	while(dwell < MAX_DWELL && abs2(z) < 2 * 2) {
		z = z * z + c;
		dwell++;
	}
	return dwell;
}


/** binary operation for common dwell "reduction": MAX_DWELL + 1 = neutral element, -1 = dwells are different */
__device__ int same_dwell(int d1, int d2) {
   if(d1 == d2)
      return d1;
   else if(d1 == NEUT_DWELL || d2 == NEUT_DWELL)
      return min(d1, d2);
   else
      return DIFF_DWELL;
}

/** evaluates the common border dwell, if it exists */
__device__ int border_dwell(int w, int h, complex cmin, complex cmax, int x0, int y0, int d) {
   // check whether all boundary pixels have the same dwell
   int tid = threadIdx.y * blockDim.x + threadIdx.x;
   int bs = blockDim.x * blockDim.y;
   int comm_dwell = NEUT_DWELL;
   // for all boundary pixels, distributed across threads
   for(int r = tid; r < d; r += bs) {
      // for each boundary: b = 0 is east, then counter-clockwise
      for(int b = 0; b < 4; b++) {
         int x = b % 2 != 0 ? x0 + r : (b == 0 ? x0 + d - 1 : x0);
         int y = b % 2 == 0 ? y0 + r : (b == 1 ? y0 + d - 1 : y0);
         int dwell = pixel_dwell(w, h, cmin, cmax, x, y);
         comm_dwell = same_dwell(comm_dwell, dwell);
      }
   }
   // reduce across threads in the block
   __shared__ int ldwells[BSX * BSY];
   int nt = min(d, BSX * BSY);
   if(tid < nt)
      ldwells[tid] = comm_dwell;
   __syncthreads();
   for(; nt > 1; nt /= 2) {
      if(tid < nt / 2)
         ldwells[tid] = same_dwell(ldwells[tid], ldwells[tid + nt / 2]);
      __syncthreads();
   }
   return ldwells[0];
}

/** the kernel to fill the image region with a specific dwell value */
__global__ void dwell_fill_k(int *dwells, int w, int x0, int y0, int d, int dwell) {
   int x = threadIdx.x + blockIdx.x * blockDim.x;
   int y = threadIdx.y + blockIdx.y * blockDim.y;
   if(x < d && y < d) {
      x += x0, y += y0;
      dwells[y * w + x] = dwell;
   }
}

/** the kernel to fill in per-pixel values of the portion of the Mandelbrot set */
__global__ void mandelbrot_pixel_k(int *dwells, int w, int h, complex cmin, complex cmax, int x0, int y0, int d) {
   int x = threadIdx.x + blockDim.x * blockIdx.x;
   int y = threadIdx.y + blockDim.y * blockIdx.y;
   if(x < d && y < d) {
      x += x0, y += y0;
      dwells[y * w + x] = pixel_dwell(w, h, cmin, cmax, x, y);
   }
}

/** computes the dwells for Mandelbrot image using dynamic parallelism; one
		block is launched per pixel
		@param dwells the output array
		@param w the width of the output image
		@param h the height of the output image
		@param cmin the complex value associated with the left-bottom corner of the
		image
		@param cmax the complex value associated with the right-top corner of the
		image
		@param x0 the starting x coordinate of the portion to compute
		@param y0 the starting y coordinate of the portion to compute
		@param d the size of the portion to compute (the portion is always a square)
		@param depth kernel invocation depth
		@remarks the algorithm reverts to per-pixel Mandelbrot evaluation once
		either maximum depth or minimum size is reached
*/
__global__ void mandelbrot_block_k(int *dwells, int w, int h, complex cmin, complex cmax, int x0, int y0, int d, int depth) {
   x0 += d * blockIdx.x, y0 += d * blockIdx.y;
   int comm_dwell = border_dwell(w, h, cmin, cmax, x0, y0, d);
   if(threadIdx.x == 0 && threadIdx.y == 0) {
      if(comm_dwell != DIFF_DWELL) {
         // uniform dwell, just fill
         dim3 bs(BSX, BSY), grid(divup(d, BSX), divup(d, BSY));
         dwell_fill_k<<<grid, bs>>>(dwells, w, x0, y0, d, comm_dwell);
      } else if(depth + 1 < MAX_DEPTH && d / SUBDIV > MIN_SIZE) {
         // subdivide recursively
         dim3 bs(blockDim.x, blockDim.y), grid(SUBDIV, SUBDIV);
         mandelbrot_block_k<<<grid, bs>>>(dwells, w, h, cmin, cmax, x0, y0, d / SUBDIV, depth	+ 1);
      } else {
         // leaf, per-pixel kernel
         dim3 bs(BSX, BSY), grid(divup(d, BSX), divup(d, BSY));
         mandelbrot_pixel_k<<<grid, bs>>>(dwells, w, h, cmin, cmax, x0, y0, d);
      }
   }
 }

/********************************************************/
/*                        MAIN                          */
/********************************************************/

/** data size */
#define H (1024)
#define W (1024)
#define IMAGE_PATH "./mandelbrot.ppm"

int main(int argc, char **argv) {
	// allocate memory
	int w = W,  h = H;
	size_t dwell_sz = w * h * sizeof(int);
	int *h_dwells, *d_dwells;
	CHECK(cudaMalloc(&d_dwells, dwell_sz));
	h_dwells = (int*)malloc(dwell_sz);

	// compute the dwells, copy them back
	dim3 bs(BSX, BSY), grid(INIT_SUBDIV, INIT_SUBDIV);
	double start = seconds();
	mandelbrot_block_k<<<grid, bs>>>(d_dwells, w, h, complex(-1.5, -1), complex(0.5, 1), 0, 0, W / INIT_SUBDIV, 1);
	CHECK(cudaDeviceSynchronize());
   double gpu_time = seconds() - start;
	CHECK(cudaMemcpy(h_dwells, d_dwells, dwell_sz, cudaMemcpyDeviceToHost));

   // print performance
	printf("Mandelbrot set computed in %.5lf s, at %.3lf Mpix/s\n", gpu_time, h * w * 1e-6 / gpu_time);


   // save the image
   PPM *ppm = ppm_make(w, h, (pel) {0,0,0});
   for (int y = 0; y < h; y++) {
		for (int x = 0; x < w; x++) {
			int r, g, b;
			dwell_color(&r, &g, &b, h_dwells[y * w + x]);
         ppm_set(ppm, x, y, (pel) {(color)r,(color)g,(color)b});
		}
   }
   ppm_write(ppm, IMAGE_PATH);
   printf("Image saved to %s\n", IMAGE_PATH);

	// free data
	cudaFree(d_dwells);
	free(h_dwells);
	return 0;
}


/** gets the color, given the dwell (on host) */
void dwell_color(int *r, int *g, int *b, int dwell) {
	// black for the Mandelbrot set
	if(dwell >= MAX_DWELL) {
		*r = *g = *b = 0;
	} else {
		// cut at zero
		if(dwell < 0)
			dwell = 0;
		if(dwell <= CUT_DWELL) {
			// from black to blue the first half
			*r = *g = 0;
			*b = 128 + dwell * 127 / (CUT_DWELL);
		} else {
			// from blue to white for the second half
			*b = 255;
			*r = *g = (dwell - CUT_DWELL) * 255 / (MAX_DWELL - CUT_DWELL);
		}
	}
}

↩ **Run...**

In [ ]:
# Compilazione ed esecuzione
!nvcc -rdc=true  -arch=sm_75 src/lez6/mandelbrot_dyn.cu -o mandelbrot_dyn -I GPUcomputing/utils/PPM GPUcomputing/utils/PPM/ppm.cpp
!./mandelbrot_dyn

### Parallelismo dinamico nel calcolo di prodotti MQDB

In [ ]:
%%cuda --name mqdb.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <sys/time.h>
#include <time.h>

#ifndef MQDB_H
#define MQDB_H

#define randu() ((float)rand() / (float) RAND_MAX)
#define abs(x) ((x)<0 ? (-x) : (x))

typedef unsigned long ulong;
typedef unsigned int uint;

typedef struct MQDB {
	char desc[100];   // description
	int nBlocks;      // num. of blocks
	int *blkSize;     // block dimensions
	float *elem;       // elements in row-major order
	ulong nElems;     // actual number of elements
} mqdb;

typedef unsigned long ulong;
typedef unsigned int uint;

// # function prototypes #
int genRandDims(mqdb*, uint, uint);
void fillBlocks(mqdb*, uint, uint, char, float);
mqdb mqdbConst(uint, uint, uint, float);
void mqdbProd(mqdb, mqdb, mqdb);
void matProd(mqdb, mqdb, mqdb);
void checkResult(mqdb, mqdb);
void mqdbDisplay(mqdb);

#endif

In [ ]:
%%cuda --name MQDB-CUDA-DP.cu

#include "mqdb.h"
#include "../GPUcomputing/utils/common.h"

#define BLOCK_SIZE 16     // block size

struct tms {
	double GPUtmsMQDB;
	double GPUtmsMQDBDynPar1;
	double GPUtmsMQDBDynPark;
	float density;
};

// the kernels prototype
__global__ void mqdbBlockProd(mqdb, mqdb, mqdb, uint, uint, uint);
__global__ void mqdbProdDP1(mqdb, mqdb, mqdb, uint, uint);
__global__ void mqdbProdDPk(mqdb, mqdb, mqdb, uint);

/*
 * Test on MQDB kernels
 */
void testKernelsMQDB(uint n, uint k, struct tms* times) {

	// mqdb host matrices
	mqdb A, B, C, C1;

	// mqdb device matrices
	mqdb d_A, d_B, d_C;

	// fill in
	A = mqdbConst(n, k, 10, 1);
	B = mqdbConst(n, k, 10, 1);
	C = mqdbConst(n, k, 10, 1);
	C1 = mqdbConst(n, k, 10, 1);

	ulong nBytes = n * n * sizeof(float);
	ulong kBytes = k * sizeof(uint);
	printf("Memory size required = %.1f (MB)\n",(float)nBytes/(1024.0*1024.0));

	// malloc and copy on device memory
	d_A.nBlocks = A.nBlocks;
	CHECK(cudaMalloc((void**)&d_A.blkSize, kBytes));
	CHECK(cudaMemcpy(d_A.blkSize, A.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_A.elem, nBytes));
	CHECK(cudaMemcpy(d_A.elem, A.elem, nBytes, cudaMemcpyHostToDevice));
	d_B.nBlocks = B.nBlocks;
	CHECK(cudaMalloc((void**)&d_B.blkSize, kBytes));
	CHECK(cudaMemcpy(d_B.blkSize, B.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_B.elem, nBytes));
	CHECK(cudaMemcpy(d_B.elem, B.elem, nBytes, cudaMemcpyHostToDevice));
	d_C.nBlocks = C.nBlocks;
	CHECK(cudaMalloc((void**)&d_C.blkSize, kBytes));
	CHECK(cudaMemcpy(d_C.blkSize, C.blkSize, kBytes, cudaMemcpyHostToDevice));
	CHECK(cudaMalloc((void**)&d_C.elem, nBytes));
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*                     GPU MQDB product                    */
	/***********************************************************/
	printf("Kernel MQDB product...\n");
	dim3 block(BLOCK_SIZE, BLOCK_SIZE);
	dim3 grid((n + block.x - 1) / block.x, (n + block.y - 1) / block.y);
	uint sdim = 0;
	double start = seconds();
	for (uint i = 0; i < k; i++ ) {
		uint d = A.blkSize[i];
		mqdbBlockProd<<<grid, block>>>(d_A, d_B, d_C, sdim, d, n);
		sdim += d;
	}
	CHECK(cudaDeviceSynchronize());
	double GPUtime2 = seconds() - start;
	printf("   elapsed time:                    %.2f (sec)\n", GPUtime2);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*              GPU MQDB dynamic par. GRID(1)              */
	/***********************************************************/
	start = seconds();
	printf("Kernel MQDB product with dynamic parall. GRID(1)...\n");
	mqdbProdDP1<<< 1, 1 >>>(d_A, d_B, d_C, k, n);
	CHECK(cudaDeviceSynchronize());
	double GPUtime3 = seconds() - start;
	printf("   elapsed time:                        %.2f (sec)\n", GPUtime3);
	printf("   speedup vs GPU MQDB product:         %.2f\n", GPUtime2/GPUtime3);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	/***********************************************************/
	/*              GPU MQDB dynamic par. GRID(k)              */
	/***********************************************************/
	start = seconds();
	printf("Kernel MQDB product with dynamic parall. GRID(k)...\n");
	mqdbProdDPk<<< 1, k >>>(d_A, d_B, d_C, n);
	CHECK(cudaDeviceSynchronize());
	double GPUtime4 = seconds() - start;
	printf("   elapsed time:                        %.2f (sec)\n", GPUtime4);
	printf("   speedup vs GPU MQDB product:         %.2f\n", GPUtime2/GPUtime4);
	printf("   speedup vs GPU MQDB product GRID(1): %.2f\n", GPUtime3/GPUtime4);
	// copy the array 'C' back from the GPU to the CPU
	CHECK(cudaMemcpy(C1.elem, d_C.elem, nBytes, cudaMemcpyDeviceToHost));
	checkResult(C,C1);
	CHECK(cudaMemset(d_C.elem, 0.0, nBytes));

	CHECK(cudaFree(d_A.elem));
	CHECK(cudaFree(d_B.elem));
	CHECK(cudaFree(d_C.elem));

	// collect times
	times->GPUtmsMQDB = GPUtime2;
	times->GPUtmsMQDBDynPar1 = GPUtime3;
	times->GPUtmsMQDBDynPark = GPUtime4;
	float den = 0;
	for (uint j = 0; j < k; j++)
		den += A.blkSize[j]*A.blkSize[j];
	times->density = den/(n*n);
}

/*
 * main function
 */
int main(int argc, char *argv[]) {
	uint n = 16*1024;      // matrix size
	uint min_k = 10;       // max num of blocks
	uint max_k = 20;       // max num of blocks

	struct tms times[max_k-min_k+1];

	// multiple tests on kernels
	for (uint k = min_k; k <= max_k; k++) {
		printf("\n*****   k = %d --- (avg block size = %f)\n",k,(float)n/k);
		testKernelsMQDB(n, k, &times[k-min_k]);
	}

	FILE *fd;
	fd = fopen("res.csv", "w");
	if (fd == NULL) {
		perror("file error!\n");
		exit(1);
	}

	// write results on file
	fprintf(fd,"num blocks,");
		for (uint j = 0; j <= max_k-min_k; j++)
			fprintf(fd,"%d,",j+min_k);

	fprintf(fd,"\nKernel MQDB product,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDB);

	fprintf(fd,"\nKernel MQDB product with dynamic parall. GRID(1),");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDBDynPar1);

	fprintf(fd,"\nKernel MQDB product with dynamic parall. GRID(k),");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].GPUtmsMQDBDynPark);

	fprintf(fd,"\ndensity,");
	for (uint j = 0; j <= max_k-min_k; j++)
		fprintf(fd,"%.4f,",times[j].density);

	fclose(fd);

	return 0;
}




↩ **Run...**

In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_70 -dc src/MQDB-CUDA-DP.cu GPUcomputing/lab1/MQDB/mqdb.cpp
!nvcc -arch=sm_70 MQDB-CUDA-DP.o mqdb.o -o mqdb_prod
!rm -rf *.o
!./mqdb_prod

In [ ]:
%%cuda --name mqdb_DP.cu

#include "../../utils/MQDB/mqdb.h"

#define BLOCK_SIZE 16     // block size


/*
 * Kernel for block sub-matrix product of mqdb
 */
__global__ void mqdbBlockProd(mqdb A, mqdb B, mqdb C, uint sdim, uint d, uint n) {
	int row = blockIdx.y * blockDim.y + threadIdx.y;
	int col = blockIdx.x * blockDim.x + threadIdx.x;

	// jump to the right block sub-matrix
	int  offset = (n+1)*sdim;

	// each thread computes an entry of the product matrix
	if ((row < d) && (col < d)) {
		float val = 0;

		for (int k = 0; k < d; k++)
			val += A.elem[row * n + k + offset] * B.elem[k * n + col + offset];
		C.elem[row * n + col + offset] = val;
	}
}

/*
 * Kernel for block sub-matrix product of mqdb: parent grid(1)
 */
__global__ void mqdbProdDP1(mqdb A, mqdb B, mqdb C, uint k, uint n) {
	// using grid(1,1)
	uint sdim = 0;
	dim3 block(BLOCK_SIZE, BLOCK_SIZE);
	dim3 grid((n + block.x - 1) / block.x, (n + block.y - 1) / block.y);
	for (uint i = 0; i < k; i++ ) {
		uint d = A.blkSize[i];
		mqdbBlockProd<<<grid, block>>>(A, B, C, sdim, d, n);
		sdim += d;
	}
}

/*
 * Kernel for block sub-matrix product of mqdb: parent grid(k)
 */
__global__ void mqdbProdDPk(mqdb A, mqdb B, mqdb C, uint n) {
	// using grid(1,k)
	uint i = threadIdx.x;
	uint sdim = 0;

	// block displacement
	uint d = A.blkSize[i];
	if (i > 0) {
		for (uint j = 0; j < i; j++)
			sdim += A.blkSize[j];
	}

	// grid dims
	dim3 grid((d + blockDim.x - 1) / blockDim.x, (d + blockDim.y - 1) / blockDim.y);
	mqdbBlockProd<<<grid,blockDim>>>(A, B, C, sdim, d, n);
}


↩ **Run...**

In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_37 -dc MQDB-CUDA-DP/mqdb_DP.cu MQDB-CUDA-DP/main.cu ../utils/MQDB/mqdb.cpp
!nvcc -arch=sm_37 mqdb_DP.o main.o mqdb.o -o mqdb_prod
!rm -rf *.o
!./mqdb_prod